In [ ]:
from LLM_things import bots
from ASP_Scheduler.problem_descriptions import all_problems
from ASP_Scheduler import scheduler
import importlib
from pprint import pprint


# If we are working locally, we just instantiate to None and by default we will call the llama 3.1 8B api
# If we work with Snellius, we can use other models from Huggingface like this
CHECKPOINT = None
# CHECKPOINT = "meta-llama/Llama-3.3-70B-Instruct"
PIPE = bots.load_pipe(model_checkpoint=CHECKPOINT, local_dir="./local_models", quantization_config='4bit')


downloading model...


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.3-70B-Instruct.
401 Client Error. (Request ID: Root=1-6808a991-4f9b01e85db70e8437aaef8a;a991ccc0-99c2-4628-9372-1098840c6675)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.3-70B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.3-70B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
# Examination Timetabling Example

# Run the LLM scheduler
full_program = scheduler.full_ASP_program(
    all_problems['examination_timetabling'],    # Input problem specifications for examination timetabling
    pipe=PIPE,                                  # Input the PIPEline object for the LLM
    printer=False)                              # Set to True to print intermediate outputs           

# Print the full program as it is returned by the scheduler
print('----------------------------FULL PROGRAM----------------------------')
print(full_program)


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


----------------------------FULL PROGRAM----------------------------

% Instance Template
exam(Exam, Duration, Is_Large).

student(Student, Exam).

period(Period, Date, Time, Duration, Is_Late, Penalty).

room(Room, Capacity, Penalty).

order_constraint(Exam1, Exam2).

same_time_constraint(Exam1, Exam2).

different_time_constraint(Exam1, Exam2).

own_room_constraint(Exam).


% Generator

1 { assigned(Exam, Period, Room) : period(Period,_,_,_,_,_), room(Room,_,_) } 1 :- exam(Exam,_,_).


% Hard Constraints

% No student sits more than one examination at the same time.
:- student(Student, Exam1), student(Student, Exam2), assigned(Exam1, Period, _), assigned(Exam2, Period, _), Exam1!= Exam2.


% The number of students taking exams in a room at the same time should not exceed the capacity of the room.
:- assigned(Exam, Period, Room), #count{Student : student(Student, Exam)} > Capacity, room(Room, Capacity, _).


% Period Lengths are not violated.
:- assigned(Exam, Period, Room), period(Per

In [ ]:
# Post Enrollment Based Course Time Tabling

full_program = scheduler.full_ASP_program(
    all_problems['post_enrollment_based_course_time_tabling'], 
    pipe=PIPE, 
    printer=False)

print('----------------------------FULL PROGRAM----------------------------')
print(full_program)

----------------------------FULL PROGRAM----------------------------

% Instance Template
event(1.. N).

room(Room, Capacity).

timeslot(Timeslot, Day, Hour).

student(Student, Event).

feature_requirement(Feature, Event).

room_feature(Feature, Room).

precedence_constraint(Event1, Event2).

timeslot_constraint(Event, Timeslot).


% Generator

1 { assigned(Event, Timeslot, Room) : room(Room, _), timeslot(Timeslot, _, _) } 1 :- event(Event).


% Hard Constraints

% Students must not have overlapping events.
:- student(Student, Event1), student(Student, Event2), assigned(Event1, Timeslot, Room), assigned(Event2, Timeslot, Room), Event1!= Event2.


% The amount of students in a room at the same time should not exceed the capacity of the room
:- room(Room, Capacity), timeslot(Timeslot, _, _), 
   #count{Student : student(Student, Event), assigned(Event, Timeslot, Room)} > Capacity.


% Rooms must have the required features for the event.
:- assigned(Event, Timeslot, Room), feature_require

In [ ]:
# Nurse Scheduling

full_program = scheduler.full_ASP_program(
    all_problems['nurse_scheduling'], 
    pipe=PIPE, 
    printer=False)

print('----------------------------FULL PROGRAM----------------------------')
print(full_program)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


----------------------------FULL PROGRAM----------------------------

% Instance Template
nurse(Nurse).

day(0.. N_days-1).

shift_type(Shift_type, Duration).

nurse_requirement(Shift, Min, Max).

work_requirement(Min, Max).

shift_requirement(Shift_type, Min, Max, Preferred).


1 { assigned(Nurse, Shift_type, Day) : shift_type(Shift_type, _), day(Day) } 1 :- nurse(Nurse), day(Day).


% Hard Constraints

% Every day, the number of nurses assigned to each shift must be between the specified minimum and maximum
:- day(Day), shift_type(Shift_type, _), #count{Nurse : assigned(Nurse, Shift_type, Day)} < Min, nurse_requirement(Shift_type, Min, _).
:- day(Day), shift_type(Shift_type, _), #count{Nurse : assigned(Nurse, Shift_type, Day)} > Max, nurse_requirement(Shift_type, _, Max).


% Each nurse must work at least the specified minimum and at most the specified maximum number of hours
:- nurse(Nurse), #sum{Duration, Shift_type, Day: assigned(Nurse, Shift_type, Day), shift_type(Shift_type, Dur